In [2]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import tqdm
import time
import warnings
warnings.filterwarnings('ignore')
import cv2
import json
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import shutil
from threading import Thread 

c:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the model
def run_example(task_prompt, image, model, processor ,text_input=None):
    prompt = task_prompt if text_input is None else task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    
    generated_ids = model.generate(
        input_ids=inputs["input_ids"].cuda(),
        pixel_values=inputs["pixel_values"].cuda(),
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )
    
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

def read_yolo_label_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        annotations = []
        for line in lines:
            # Split the line into components
            parts = line.strip().split()
            # Convert each part to a float (or int for class_id)
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            # Append the annotation to the list
            annotations.append([class_id, x_center, y_center, width, height])
    return annotations

def plot_bbox(original_image, bboxes, labels, title):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(original_image)
    axs[0].set_title('Original Image')
    axs[0].axis('off')
    
    axs[1].imshow(original_image)
    
    for bbox, label in zip(bboxes, labels):
        x1, y1, x2, y2 = bbox
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
        axs[1].add_patch(rect)
        axs[1].text(x1, y1, label, color='white', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))
    
    axs[1].set_title(title)
    axs[1].axis('off')
    plt.show(block=False)
    # plt.pause(1)
    # Close the plot
    plt.close()

def provide_yolo_annotations(annotations, img_height,img_width):
    xyxy_annotations = []
    labels_annotations = []
    for annotation in annotations:
        class_id, x_center, y_center, iwidth, iheight = annotation
        x1 = x_center - iwidth/2
        x2 = x_center + iwidth/2
        y1 = y_center - iheight/2
        y2 = y_center + iheight/2
        xyxy_annotations.append([x1*img_width,y1*img_height,x2*img_width,y2*img_height])
        labels_annotations.append(class_id)
    return xyxy_annotations,labels_annotations

def save_yolo_annotations(data_path,image_path,bboxes,labels,img_height,img_width):
    base_name = os.path.basename(image_path)
    annotations_dir = os.path.join(data_path,"labels")
    if not os.path.exists(annotations_dir):
        os.mkdir(annotations_dir)
    annotations_file_name = os.path.splitext(base_name)[0]+".txt"
    label_path = os.path.join(annotations_dir,annotations_file_name)
    print(label_path)
    with open(label_path, 'w') as file:
        count = 0
        for bbox,label in zip(bboxes,labels):
            count = count + 1
            width = bbox[2]-bbox[0]
            height = bbox[3]-bbox[1]
            x_center = (bbox[0]+width/2)/img_width
            y_center = (bbox[1]+height/2)/img_height
            width = width/img_width
            height = height/img_height
            string = None
            if count==1:
                string = "{} {} {} {} {}".format(crud_label_map(data_path,label), x_center,y_center,width,height)
            elif count>1:
                string = "\n{} {} {} {} {}".format(crud_label_map(data_path,label), x_center,y_center,width,height)
            file.write(string)
    return label_path

def crud_label_map(data_path,label):
    base_path = os.path.join(data_path,"label_map.json")
    if os.path.exists(base_path) == False:
        with open(base_path, 'w') as file:
            json.dump({}, file)
    with open(base_path, 'r') as file:
        labels_map = json.load(file)
    if label in labels_map:
        return labels_map[label]
    else:
        label_count = len(labels_map)
        labels_map[label] = label_count
        with open(base_path, 'w') as file:
            json.dump(labels_map, file)
        return labels_map[label]

def train_val_test_split(data_dir,split_map={"train":0.7,"valid":0.2,"test":0.1}):
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'valid')
    test_dir = os.path.join(data_dir, 'test')
    
    os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)
    # Split the data into training and validation sets
    images = os.listdir(images_dir)
    labels = os.listdir(labels_dir)
    print("Number of Images is {} and Labels is {}".format(len(images),len(labels)))
    train_images, test_val_images = train_test_split(images, test_size=split_map["valid"]+split_map["test"])
    test_val_split = split_map["test"]/(split_map["test"]+split_map["valid"])
    valid_images, test_images = train_test_split(test_val_images, test_size=round(test_val_split,1))

    for image in train_images:
        if os.path.exists(os.path.join(labels_dir, os.path.splitext(image)[0]+'.txt')):
            shutil.copy(os.path.join(images_dir, image), os.path.join(train_dir, 'images'))
            shutil.copy(os.path.join(labels_dir, os.path.splitext(image)[0]+'.txt'), os.path.join(train_dir, 'labels'))
    for image in valid_images:
        if os.path.exists(os.path.join(labels_dir, os.path.splitext(image)[0]+'.txt')):
            shutil.copy(os.path.join(images_dir, image), os.path.join(val_dir, 'images'))
            shutil.copy(os.path.join(labels_dir, os.path.splitext(image)[0]+ '.txt'), os.path.join(val_dir, 'labels'))
    for image in test_images:
        if os.path.exists(os.path.join(labels_dir, os.path.splitext(image)[0]+'.txt')):
            shutil.copy(os.path.join(images_dir, image), os.path.join(test_dir, 'images'))
            shutil.copy(os.path.join(labels_dir, os.path.splitext(image)[0]+ '.txt'), os.path.join(test_dir, 'labels'))

    # shutil.rmtree(images_dir)
    # shutil.rmtree(labels_dir)

def create_yaml_file(base_path):
    base_path = os.path.join(base_path,"label_map.json")
    with open(base_path, 'r') as file:
        labels_map = json.load(file)
    # Define the yaml content
    yaml_content = f"""
train: train/images
val: valid/images
test: test/images  # Optional, only if you have a separate test dataset

nc: {len(labels_map)}  # Number of classes
names:
"""
    for label, idx in labels_map.items():
        yaml_content += f"  {idx}: '{label}'  # Class name for label {idx}\n"

    # Write to coco.yaml
    with open("coco8.yaml", "w") as yaml_file:
        yaml_file.write(yaml_content)
    pass

def yolo_trainer():
    model_yolo = YOLO('yolo11n.pt')  # Specify the correct YOLOv11 config file
    results = model_yolo.train(data="coco8.yaml", epochs=100, imgsz=640)
    return results

def run_annotaion_tool(data_path,task_prompt,text_input,model_id):
    model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    for path_img in os.listdir(path=os.path.join(data_path,"images")):
        if path_img.endswith(".jpg"):
            image_path = os.path.join((os.path.join(data_path,"images")),path_img)
            cv2_image = cv2.imread(image_path)
            # cv2_image.resize((640,640,3))
            # cv2.imwrite(image_path,cv2_image)
            img = Image.open(image_path)
            img_height,img_width = cv2_image.shape[:2]
            print("image read : {} \nwidth : {}, height : {}".format(image_path,img_width,img_height))
            print("run example called")
            results = run_example(task_prompt, img, model, processor, text_input=text_input)
            data = results['<CAPTION_TO_PHRASE_GROUNDING>']
            bboxes,labels = data['bboxes'],data['labels']
            print("bboxes : {}\nlabel : {}".format(bboxes,labels))
            # plot_bbox(img,bboxes,labels,"Image From Florence-2")
            print("save yolo annotations called")
            label_path = save_yolo_annotations(data_path,image_path,bboxes,labels,img_height,img_width)
            print("annotations returned path : {}".format(label_path))
            annotations = read_yolo_label_file(label_path)
            xyxy_annotations,labels_annotations = provide_yolo_annotations(annotations, img_height,img_width)
            # plot_bbox(img, xyxy_annotations, labels_annotations,"Image From Yolo Label file")
    return True

In [4]:
model_id = 'microsoft/Florence-2-large-ft'
data_path = r"C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets"
task_prompt = '<CAPTION_TO_PHRASE_GROUNDING>'

text_input = "a license plate"

In [5]:
run_annotaion_tool(data_path,task_prompt,text_input,model_id)

Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


image read : C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\images\1220841d1395058453-take-look-number-plate-img_2557.jpg 
width : 597, height : 800
run example called
bboxes : [[184.77149963378906, 487.6000061035156, 365.06549072265625, 541.2000122070312]]
label : ['a license plate']
save yolo annotations called
C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\labels\1220841d1395058453-take-look-number-plate-img_2557.txt
annotations returned path : C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\labels\1220841d1395058453-take-look-number-plate-img_2557.txt
image read : C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\images\16855_7ae98c41cef74ccd898643340a8d56b6IMG_20230106_141542.jpg 
width : 331, height : 450
run example called
bboxes : [[0.1655000001192093, 11.47499942779541, 330.5035095214844, 435.375]]
label

True

In [6]:
train_val_test_split(data_path)
create_yaml_file(data_path)
yolo_trainer()

Number of Images is 124 and Labels is 60
New https://pypi.org/project/ultralytics/8.3.13 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.12  Python-3.12.5 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_

train: Scanning C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\train\labels... 56 images, 0 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<00:00, 1700.13it/s]

train: New cache created: C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\train\labels.cache



val: Scanning C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\valid\labels... 22 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<00:00, 1222.13it/s]

val: New cache created: C:\Users\nikhi\Desktop\Projects\LLM activity\LLM test 1\Florence_2_training_yolo11\datasets\valid\labels.cache


...

                   all         22         25      0.997       0.84      0.906      0.813



100 epochs completed in 0.079 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 5.5MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.12  Python-3.12.5 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.74it/s]


                   all         22         25      0.956      0.873      0.905      0.818
Speed: 0.3ms preprocess, 1.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002970D19E0C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [1]:
!pip install flask

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached werkzeug-3.0.4-py3-none-any.whl.metadata (3.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.0.4-py3-none-any.whl (227 kB)
